# AIC23

Load dataset and launch app

In [1]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob
import numpy as np

dataset = fo.Dataset.from_images_patt("./data/keyframes/*/*.jpg", name=None, tags=None)
session = fo.launch_app(dataset, desktop=False)
session.wait()

 100% |█████████████| 72410/72410 [6.1s elapsed, 0s remaining, 11.9K samples/s]      


Notebook sessions cannot wait


In [2]:
import os
import csv
import pandas as pd

info = dict()
for file in os.scandir("map-keyframes"):
    if file.name[0] == '.':
        continue
    videoId = file.name.split(".")[0]
    data = pd.read_csv(file.path)
    info[videoId] = data.to_dict()

In [3]:
import json

for sample in dataset:
    if "checkpoint" in sample["filepath"]:
        continue;
    videoId, frameId = sample["filepath"].split("/")[-2:]
    videoId = videoId.split(".")[0]
    frameId = frameId.split(".")[0]
    object_file = f"./objects/{videoId}/{frameId}.json"
    frameId = int(frameId)
    sample["videoId"] = videoId
    sample["n"] = str(info[videoId]['n'][frameId-1])
    sample["frameId"] = str(info[videoId]['frame_idx'][frameId-1])
    sample["pts_time"] = str(info[videoId]['pts_time'][frameId-1])

    f = open(object_file)
    data = json.load(f)
    detections = []
    for id in range(len(data["detection_class_entities"])):
        if float(data["detection_scores"][id]) < 0.5:
            continue;
        detections.append(
            fo.Detection(
                label=data["detection_class_entities"][id],
                bounding_box=data["detection_boxes"][id],
                confidence=data["detection_scores"][id],
            )
        )
    sample["prediction"] = fo.Detections(detections=detections)
    sample.save()

### Embeddings of videos are provided 

Compute embeddings (if not provided)

In [ ]:
model = foz.load_zoo_model("clip-vit-base32-torch")
embeddings = dataset.compute_embeddings(model)
with open("./embeddings/keyframes.npy", "wb") as f:
    np.save(f, embeddings);

Load embeddings

In [4]:
embeddings = np.load("./embeddings/keyframes.npy")

Compute visualization using embeddings

In [5]:
results = fob.compute_visualization(
    dataset, 
    embeddings=embeddings, 
    seed=51, 
    brain_key="img_viz"
)

Generating visualization...


/Users/namkhanhtran/programming/repo/AIC23/env/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/namkhanhtran/programming/repo/AIC23/env/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/namkhanhtran/programming/repo/AIC23/env

UMAP(random_state=51, verbose=True)
Thu Sep 28 13:09:18 2023 Construct fuzzy simplicial set
Thu Sep 28 13:09:18 2023 Finding Nearest Neighbors
Thu Sep 28 13:09:18 2023 Building RP forest with 18 trees
Thu Sep 28 13:09:20 2023 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	 4  /  16
	 5  /  16
	Stopping threshold met -- exiting after 5 iterations
Thu Sep 28 13:09:26 2023 Finished Nearest Neighbor Search
Thu Sep 28 13:09:27 2023 Construct embedding


Epochs completed: 100%| █████████████████████████████████████████████████████████████████████████████████████ 200/200 [00:27]


Thu Sep 28 13:10:05 2023 Finished embedding


Compute similarity (used to sort by similarity or query by text)

In [6]:
image_index = fob.compute_similarity(
    dataset,
    model="clip-vit-base32-torch", 
    embeddings=embeddings,       
    brain_key="img_sim",
)